In [1]:
import pandas as pd
! pip install transformers evaluate rouge_score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import evaluate
from datasets import load_metric
import numpy as np
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 K

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def preprocess_data(data): #this is used to process the data for the tokenizer
    context = data['context'].to_list() #First convert to a list
    text_encodings = tokenizer(context, truncation=True, padding=True)

    triplets = data['triplets'].to_list()
    label_encodings = tokenizer(triplets, truncation=True, padding=True)
    #new_data ={"input_ids":model_inputs["input_ids"], "labels": labels["input_ids"]}

    return text_encodings, label_encodings

class RebelDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])

        return item

    def __len__(self):
        return len(self.labels['input_ids'])


In [3]:
model_checkpoint = "Babelscape/rebel-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
seed = 1
data = pd.read_csv('rebel_format.csv')
train_data, val_data = train_test_split(data, test_size=0.2, random_state=seed)
del data

train_encodings, train_labels = preprocess_data(train_data)
train_data = RebelDataset(train_encodings, train_labels)
val_encodings, val_labels = preprocess_data(val_data)
val_data = RebelDataset(val_encodings, val_labels)

In [5]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-faro-relations",
    evaluation_strategy = "epoch",
    learning_rate=0.000025,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.1,
    save_total_limit=3,
    save_steps=300,
    #load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    num_train_epochs=7,
    predict_with_generate=True,
    fp16 = True,
    push_to_hub=False,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [6]:
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

<ipython-input-6-77060938401b>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [7]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics= compute_metrics
)

trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1261
  Num Epochs = 7
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1106
  Number of trainable parameters = 406298624
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.481464,62.247600,42.174300,62.149000,62.113900,43.531600
2,No log,0.376713,73.817600,55.448300,73.685000,73.801900,12.519000
3,No log,0.373409,76.999000,56.705100,77.001500,77.036400,10.420900
4,0.958200,0.380222,74.851500,56.174700,74.779200,74.972000,10.205700
5,0.958200,0.445893,76.056500,56.231400,75.786400,75.945100,10.474700
6,0.958200,0.460927,77.007700,57.776900,76.680600,76.885000,10.525300
7,0.108300,0.449298,77.429800,58.260200,77.164900,77.386200,10.408200


***** Running Evaluation *****
  Num examples = 316
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1

TrainOutput(global_step=1106, training_loss=0.48981213957664116, metrics={'train_runtime': 865.0303, 'train_samples_per_second': 10.204, 'train_steps_per_second': 1.279, 'total_flos': 2783423648538624.0, 'train_loss': 0.48981213957664116, 'epoch': 7.0})

In [8]:
trainer.save_model("finetuned_rebel")

Saving model checkpoint to finetuned_rebel
Configuration saved in finetuned_rebel/config.json
Configuration saved in finetuned_rebel/generation_config.json
Model weights saved in finetuned_rebel/pytorch_model.bin
tokenizer config file saved in finetuned_rebel/tokenizer_config.json
Special tokens file saved in finetuned_rebel/special_tokens_map.json


In [9]:
import shutil
shutil.make_archive("rebel_finetuned", 'zip', "finetuned_rebel")

'/content/rebel_finetuned.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
shutil.unpack_archive('/content/drive/MyDrive/rebel_finetuned.zip', '/content/rebel_finetuned', 'zip')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("rebel_finetuned")
tokenizer = AutoTokenizer.from_pretrained('Babelscape/rebel-large')

In [ ]:
text = ["because the machine is old, it is unreliable", "many people have died in the storm", "now the preparation is complete, we can start again",
        "the restrictions made sure less people got infected", "I am running everyday because i want to run a marathon", "the elevator is fixed, so i can go up again",
        "There was a traffic jam, so i was late", "I broke my leg, so I can't run the marathon", "Since I failed the exam, I can't graduate",
        "I did some shopping, because i want to cook later", "I shouldn't have said that, I did not mean that", "I wanted to say that", "I am planning on doing that later",
        "I intend on doing that"]
encoding = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# forward pass
outputs = model.generate(**encoding, do_sample=True)
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=False)
print(decoded_output)

In [ ]:
for t, o in zip(text, decoded_output):
  print(f"{t} - {o}")